In [12]:
import pandas as pd
import cudf
from numba import jit

In [13]:
nanodoc_16s = cudf.read_csv("/home/bhargavam/Documents/data/ndoc_final/ndoc_16s_1.txt", sep='\t')
nanodoc_16s.columns = ['position', '5mer', 'depth_tgt', 'depth_ref', 'med_current', 'mad_current', 'med_currentR', 'mad_currentR', 'current_ratio', 'scoreSide1', 'scoreSide2', 'score']

print(nanodoc_16s.head())
print(nanodoc_16s.shape)

   position   5mer  depth_tgt  depth_ref  med_current  mad_current  \
0         2  GAGGA          3          3          0.0          0.0   
1         3  AGGAA          3          3          0.0          0.0   
2         4  GGAAA          4          4          0.0          0.0   
3         5  GAAAG          4          4          0.0          0.0   
4         6  AAAGC          4          4          0.0          0.0   

   med_currentR  mad_currentR  current_ratio  scoreSide1  scoreSide2  score  
0           0.0           0.0            0.0         0.0         0.0    0.0  
1           0.0           0.0            0.0         0.0         0.0    0.0  
2           0.0           0.0            0.0         0.0         0.0    0.0  
3           0.0           0.0            0.0         0.0         0.0    0.0  
4           0.0           0.0            0.0         0.0         0.0    0.0  
(1936, 12)


In [14]:
w = 1

In [15]:
known_mods_16s = [716, 727, 1166, 1167, 1407, 1602, 1607, 1698, 1716, 1718, 1719]
known_mods_16s_pos = [x+w for x in known_mods_16s]
known_mods_16s_neg = [x-w for x in known_mods_16s]

In [18]:
@jit
def conditions_16s(a):
    if a in (known_mods_16s + known_mods_16s_pos + known_mods_16s_neg):
        return 1
    else:
        return 0

In [19]:
nanodoc_16s['mods'] = nanodoc_16s['position'].apply(conditions_16s)

ValueError: user defined function compilation failed.

In [7]:
## define function to generalize the CSV generation for ROC curve generation for 16s rRNA

def roc(filename, w):
    # read in the output file from nanodoc analysis for 16s rRNA
    
    nanodoc_16s = cudf.read_csv(filename, sep='\t')
    nanodoc_16s.columns = ['position', '5mer', 'depth_tgt', 'depth_ref', 'med_current', 'mad_current', 'med_currentR', 'mad_currentR', 'current_ratio', 'scoreSide1', 'scoreSide2', 'score']

    ## calculate values for ROC for 16s rRNA - add mods column with either yes for known modifications or no for all other positions

    known_mods_16s = [716, 727, 1166, 1167, 1407, 1602, 1607, 1698, 1716, 1718, 1719]
    known_mods_16s_pos = [x+w for x in known_mods_16s]
    known_mods_16s_neg = [x-w for x in known_mods_16s]

    def conditions_16s(a):
        if a in (known_mods_16s + known_mods_16s_pos + known_mods_16s_neg):
            return 1
        else:
            return 0
        
    nanodoc_16s['mods'] = nanodoc_16s['position'].apply(conditions_16s)

    ## sort the rows based on the log values

    sorted_16s = nanodoc_16s.sort_values('score')

    ## calculate variables required for ROC visualization

    thresholds = list(abs(sorted_16s['score']))

    roc_point = []

    for threshold in thresholds:
            
        tp = 0; fp = 0; fn = 0; tn = 0

        for index, instance in sorted_16s.iterrows():
            actual = instance["mods"]
            prediction = abs(instance["score"])

            #print(actual, prediction)
            
            if prediction >= threshold:
                prediction_class = 1
            else:
                prediction_class = 0

            if prediction_class == 1 and actual == 1:
                tp = tp + 1
            elif actual == 1 and prediction_class == 0:
                fn = fn + 1
            elif actual == 0 and prediction_class == 1: 
                fp = fp + 1
            elif actual == 0 and prediction_class == 0:
                tn = tn + 1

        #print(tp, fp, fn, tn)

        tpr = tp / (tp + fn)
        fpr = fp / (tn + fp)

        #print(tpr, fpr)

        roc_point.append([tpr, fpr])
    
    ## write out ROC values into a dataframe

    pivot = cudf.DataFrame(roc_point, columns=["tpr", "fpr"])
    pivot["threshold"] = thresholds
    pivot.to_csv(filename + "_roc_" + str(w) + "nt_window.tsv")
    
    return

if __name__ == '__main__':
  fire.Fire(roc)


ERROR: The function received no value for the required argument: w
Usage: ipykernel_launcher.py FILENAME W

For detailed information on this command, run:
  ipykernel_launcher.py --help


FireExit: 2

/home/bhargavam/mambaforge/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
